In [1]:
# import libraries
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)
import time
import json

We are going to read the energy eigenvalues and dipole matrix elements from .npy files.  We will also still create an instance of the PFHamiltonianGenerator class so we can use its build_pcqed_pf_hamiltonian() method, but it is not really important what details we use to instantiate this class... so we will use LiH in a minimal basis since this is a fast way to instantiate the class!

In [2]:
# read data from .npy files for formaldehyde casci(8,8) calculations

# !!! Change this to the correct path on your computer!
npy_folder = "/Users/rmandern/code/SCQED-PCQED/Formaldehyde/"

# these file names should still be good
E_npy_file = npy_folder + "CH2O_ccpVDZ_CASCI_88_1000_dipole_allowed_E_Array.npy"
Mu_npy_file = npy_folder + "CH2O_ccpVDZ_CASCI_88_1000_dipole_allowed_Mu_Array.npy"

# store energy eigenvalues in E_array
E_array = np.load(E_npy_file)
# store dipole matrix elements in Mu_array
Mu_array = np.load(Mu_npy_file)

# print their shape so we know how many elements we have
print(np.shape(E_array))
print(np.shape(Mu_array))

# setup basic arguments to create an instance of the PFHamiltonianGenerator class
mol_str = """
Li
H 1 1.8
symmetry c1
"""

options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}


cavity_free_dict = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0.0]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 0, 
}

# create the instance of our PFHamiltonianGenerator class
instance = PFHamiltonianGenerator(mol_str, options_dict, cavity_free_dict)

(1000,)
(1000, 1000, 3)

Start SCF iterations:

Canonical RHF One-electron energy = -12.2195250859903002
CQED-RHF One-electron energy      = -12.2195250859903002
Nuclear repulsion energy          = 0.8819620177833333
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -7.8500186970978660   dE = -7.85002E+00   dRMS = 4.96348E-15
SCF Iteration   2: Energy = -7.8500186970978589   dE =  7.10543E-15   dRMS = 1.63539E-15
Total time for SCF iterations: 0.001 seconds 

QED-RHF   energy: -7.85001870 hartree
Psi4  SCF energy: -7.85001870 hartree
 Completed QED-RHF in 0.31906890869140625 seconds
 Completed 1HSO Build in 6.890296936035156e-05 seconds
 Completed ERI Build in 0.0012531280517578125 seconds 
 Completed 2D build in 0.00011014938354492188 seconds
 Completed 1G build in 2.002716064453125e-05 seconds
 Completed the Dipole Matrix Build in 4.1961669921875e-05 seconds
 Completed determinant list in 0.0004990100860595703 seconds 
 Completed constant offset matri

In [3]:
# open json file
f = open("CH2O_CO_r_1.3_casci_cc-pVDZ_n_act_els_8_n_act_orbs_8_dipole_allowed.json")
data = json.load(f)

In [4]:
# get the array of lambda values from the json file
lambdas = data["model"]["lambda_vector"]

# determine how many lambda values will be in the scan
N_l = len(lambdas)
print(N_l)

11


In [5]:
fast_start = time.time()
# instance.fast_build_pcqed_pf_hamiltonian(N_el, N_ph, omega, lambda_vector, E_array, Mu_array)
fast_end = time.time()
print(F"Fast build took {fast_end-fast_start} seconds")

Fast build took 5.2928924560546875e-05 seconds


In [6]:
# add lambda values from jason file 
# cal energy by looping and save them agian in jason file 



N_el = 5
N_ph = 2
omega = 0.12086
lambda_vector = np.array([0., 0., 0.05])

fast_start = time.time()
# create an array of zeros to store the PCQED eigenvalues for each value of lambda
# _pcqed_52 = np.zeros((N_l, N_el * N_ph))
# loop over values of lambda, build Hamiltonian, capture eigenvalues
ctr = 0
for lam in lambdas:
    _HPF = instance.fast_build_pcqed_pf_hamiltonian(N_el, N_ph, omega, lambda_vector, E_array, Mu_array)
#     pf_e, pf_c = np.linalg.eigh(_HPF)
#     _pcqed_52[ctr, :] = pf_e
    ctr += 1
# instance.PCQED_HF_PF
E_pcqed_52 = instance.PCQED_eigs
# instance.PCQED_vecs
print(E_pcqed_52)
fast_end = time.time()
print(F"Fast build took {fast_end-fast_start} seconds")

[-113.8924874  -113.76952301 -113.55666866 -113.5555091  -113.53401732
 -113.48042166 -113.4348674  -113.43258368 -113.41271348 -113.35812345]
Fast build took 0.005110025405883789 seconds


In [7]:
N_el = 200
N_ph = 2
omega = 0.12086
lambda_vector = np.array([0., 0., 0.05])

fast_start = time.time()
# create an array of zeros to store the PCQED eigenvalues for each value of lambda
# _pcqed_52 = np.zeros((N_l, N_el * N_ph))
# loop over values of lambda, build Hamiltonian, capture eigenvalues
ctr = 0
for lam in lambdas:
    _HPF = instance.fast_build_pcqed_pf_hamiltonian(N_el, N_ph, omega, lambda_vector, E_array, Mu_array)
#     pf_e, pf_c = np.linalg.eigh(_HPF)
#     _pcqed_52[ctr, :] = pf_e
    ctr += 1
# instance.PCQED_HF_PF
E_pcqed_2002 = instance.PCQED_eigs
# instance.PCQED_vecs
print(E_pcqed_2002)
fast_end = time.time()
print(F"Fast build took {fast_end-fast_start} seconds")

[-113.88900539 -113.76656995 -113.55473153 -113.5521697  -113.53082661
 -113.47778063 -113.46857875 -113.45606902 -113.45208029 -113.43318512
 -113.42825946 -113.40569451 -113.36606084 -113.36218698 -113.35619457
 -113.34805251 -113.34281955 -113.33560458 -113.33292278 -113.32599794
 -113.29315999 -113.27331183 -113.26794417 -113.26342571 -113.24666147
 -113.24205279 -113.23578739 -113.2308336  -113.22772177 -113.22590578
 -113.21757688 -113.20945267 -113.20852844 -113.17872723 -113.16900112
 -113.16207871 -113.15369566 -113.14712402 -113.14420364 -113.14254337
 -113.12713796 -113.12084933 -113.11664877 -113.10600811 -113.09980051
 -113.09872156 -113.0943906  -113.08985207 -113.08968326 -113.08578042
 -113.0808879  -113.05297975 -113.04363434 -113.04253789 -113.04190282
 -113.02713958 -113.02276048 -113.02142901 -113.02036296 -113.01187911
 -113.00743988 -113.00305837 -112.99757011 -112.99553392 -112.99421294
 -112.97150953 -112.97103727 -112.96971592 -112.95887322 -112.9587519
 -112.9

In [8]:
N_el = 500
N_ph = 2
omega = 0.12086
# lambda_vector = np.array([0., 0., 0.05])

fast_start = time.time()
# create an array of zeros to store the PCQED eigenvalues for each value of lambda
# _pcqed_52 = np.zeros((N_l, N_el * N_ph))
# loop over values of lambda, build Hamiltonian, capture eigenvalues
ctr = 0
for lam in lambdas:
    _HPF = instance.fast_build_pcqed_pf_hamiltonian(N_el, N_ph, omega, lambda_vector, E_array, Mu_array)
#     pf_e, pf_c = np.linalg.eigh(_HPF)
#     _pcqed_52[ctr, :] = pf_e
    ctr += 1
# instance.PCQED_HF_PF
E_pcqed_5002 = instance.PCQED_eigs
# instance.PCQED_vecs
print(E_pcqed_5002)
fast_end = time.time()
print(F"Fast build took {fast_end-fast_start} seconds")

[-113.88900414 -113.76656763 -113.55472459 -113.5521473  -113.53081314
 -113.47776708 -113.46856898 -113.45605135 -113.45205336 -113.43317501
 -113.42823989 -113.40567271 -113.36605695 -113.36215471 -113.35617391
 -113.34802274 -113.34280944 -113.33558004 -113.33289313 -113.32595736
 -113.29311278 -113.27327488 -113.2679045  -113.26338731 -113.24664053
 -113.24204212 -113.23575278 -113.23081375 -113.22767045 -113.22588022
 -113.21751872 -113.20941827 -113.20846451 -113.17806451 -113.16895798
 -113.16202576 -113.15362178 -113.14670792 -113.14413797 -113.14249743
 -113.12662371 -113.12083119 -113.11660045 -113.10598851 -113.09974594
 -113.09861501 -113.09355217 -113.089505   -113.08839051 -113.08570037
 -113.08040019 -113.05245036 -113.04289218 -113.04234985 -113.04174686
 -113.02620677 -113.02225461 -113.0210404  -113.01948224 -113.01097632
 -113.00658732 -113.00263129 -112.99691029 -112.99476376 -112.99407392
 -112.97108364 -112.97024289 -112.9691733  -112.9585885  -112.95795788
 -112.

# Notes on the matrix blocks

\begin{align}\label{EQN:projected_matrix}
{\bf \mathcal{H}} =
&\begin{bmatrix}
{\bf E} + {\bf D}   & -\sqrt{\frac{\omega}{2}} {\bf d}  & 0 & \dots & 0 & 0 \\
-\sqrt{\frac{\omega}{2}}{\bf d} & {\bf E} + {\bf D} + {\bf \Omega}   & -\sqrt{\omega} {\bf d}  & \dots & 0 & 0 \\
0   &  -\sqrt{\omega}{\bf d} & {\bf E} + {\bf D} + 2{\bf \Omega} & \dots & 0  & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0      &      0 &   0    &  \dots     & {\bf E} + {\bf D} + (N-1){\bf \Omega}    & -\sqrt{\frac{N \omega}{2}} {\bf d} \\
0      &      0 &   0    &  \dots     & -\sqrt{\frac{N \omega}{2}}{\bf d}     & {\bf E} + {\bf D} + N{\bf \Omega}. 
\end{bmatrix}
\end{align}


Note that there are basically 4 different matrices with shape $(N_{el}, N_{el})$ that appear in this Hamiltonian matrix.  The following outlines the equations for the elements of each matrix along with one or more strategies to assemble them.  

### E matrix
#### Math
$$ E_{\alpha \beta} = \langle \psi_{\alpha} | \mathcal{H}_{el} | \psi_{\beta} \rangle = E_{\beta} \delta_{\alpha \beta} $$ 
where $E_{\beta}$ denote the electronic energy eigenvalues of the molecular system.
#### Code
If the energy eigenvalues are stored in an array called `E_array`, then we can build ${\bf E}$ by multiplying these values by an $(N_{el}, N_{el})$ identity matrix $\mathbb{I}$.

##### build N_el x N_el identity matrix
`_I = np.eye(N_el)`

##### build N_el x N_el _E matrix
`_E = E_array * _I`




In [ ]:
# code to build E matrix goes here
_I = np.eye(N_el)
_A = E_array[:N_el] * _I

### $\Omega$ matrix
#### Math
$$ \Omega_{\alpha \beta} = \langle \psi_{\alpha} | \omega | \psi_{\beta} \rangle  = \omega \delta_{\alpha \beta}$$ 

#### Code
If the photon frequency is stored in the variable `omega`, then the $(N_{el},N_{el})$ matrix `_O` can be build 
by multiplying the $\mathbb{I}$ by $omega$:

##### build N_el x N_el _O matrix
`_O = omega * _I`


In [ ]:
# code to build Omega matrix goes here
_O = omega * _I

### d matrix
#### Math
$$ d_{\alpha \beta} = \lambda \cdot  \langle \psi_{\alpha} | \mathcal{\mu} | \psi_{\beta} \rangle = \lambda_x \mathcal{\mu}_{x, \alpha \beta} +  \lambda_y \mathcal{\mu}_{y, \alpha \beta} +  \lambda_z \mathcal{\mu}_{z, \alpha \beta} $$ 
where $\mathcal{\mu}_{x, \alpha \beta}$ denotes the x-component of the (transition) dipole moment between molecular electronic state $\psi_{\alpha}$ and $\psi_{\beta}$.
#### Code
If the dipole matrix elements are stored in a $(N_{el}, N_{el}, 3)$  array called `mu_array` and the $\lambda$ vector is stored in an 3-element array called `\lambda_vector`, then we can build the
$(N_{el}, N_{el})$ array ${\bf d}$ by [contraction](https://en.wikipedia.org/wiki/Tensor_contraction) using `np.einsum()`:

##### build N_el x N_el _d matrix
`_d = np.einsum("k,ijk->ij", lambda_vector, mu_array)`

In [ ]:
# code to build the d matrix goes here
_d = np.einsum("k,ijk->ij", lambda_vector, Mu_array)

### D matrix
#### Math
$$ D_{\alpha \beta} = \frac{1}{2} \sum_{\gamma} d_{\alpha \gamma} d_{\gamma \beta} $$ 

#### Code
We have the $(N_{el}, N_{el})$ elements of the ${\bf d}$ array stored in `_d`.  We can then build `_D`
using matrix-matrix multiplication as follows:

`_D = _d @ _d`

or using einsum as follows:

`_D = np.einsum("ik,kj->ij",_d, _d)`


In [ ]:
# code to build the D matrix goes here
_D = _d @ _d
# _D = np.einsum("ik,kj->ij",_d, _d)